# Scrapy tutorial
Author: Jaren Haber <br>
Institution: UC Berkeley <br>
Date: March 20, 2019 <br>
Purpose: PS239T Guest lecture <br>
Source credit: I borrow heavily from tutorials by [scrapy](https://docs.scrapy.org/en/latest/intro/tutorial.html) (especially) and also [Digital Ocean](https://www.digitalocean.com/community/tutorials/how-to-crawl-a-web-page-with-scrapy-and-python-3).

## Introduction
Web scraping, often called web crawling or web spidering, or “programatically going over a collection of web pages and extracting data,” is a powerful tool for working with data on the web. With a web scraper, you can mine data about a set of products, get a large corpus of text or quantitative data to play around with, get data from a site without an official API, or just satisfy your own personal curiosity.

In this tutorial, you’ll learn about the fundamentals of the webscraping process as you explore [quotes.toscrape.com](quotes.toscrape.com), a scraping-friendly website that lists quotes from famous authors. By the end of this tutorial, you’ll have a fully functional Python web scraper that walks through a series of pages and extracts data from each page. The scraper will be easily expandable so you can tinker around with it and use it as a foundation for your own projects scraping data from the web.

Scraping has several steps:
- find web pages and download them (called *web-crawling*)
- extract information from these pages (usually with CSS or XPath)
- store formatted results to disk

All these steps can be implemented in a number of ways in many languages. You can build a scraper from scratch using low-level modules or libraries, but then you have to deal with some potential headaches as your scraper grows more complex. For example, you'll need to handle concurrency so you can crawl more than one page at a time. You'll probably want to figure out how to transform your scraped data into different formats like CSV, XML, or JSON. And you'll sometimes have to deal with sites that require specific settings and access patterns.

You'll have better luck if you build your scraper on top of an existing library that handles those issues for you. For this tutorial, we're going to use Python and Scrapy to build our scraper. *Scrapy* is one of the most popular and powerful Python scraping libraries; it takes a "batteries included" approach to scraping, meaning that it handles a lot of the common functionality that all scrapers need so developers don't have to reinvent the wheel each time. 

The focus here is on applying Scrapy--arguably the most useful, flexible current tool for web-scraping. You can also read more about the [basics of scrapy](https://docs.scrapy.org/en/latest/intro/overview.html), read about its [architecture](https://docs.scrapy.org/en/latest/topics/architecture.html), or see [the FAQ](https://docs.scrapy.org/en/latest/faq.html). And if you need a refresher on scraping with Beautiful Soup, here's a [good tutorial](https://www.digitalocean.com/community/tutorials/how-to-scrape-web-pages-with-beautiful-soup-and-python-3).

**Learning goals of this tutorial**:

You will have basic skill and understanding of:<br><br>

<li> Creating a new Scrapy project
<li> Writing a spider to crawl a site and extract data
<li> Exporting the scraped data using the command line
<li> Changing spider to recursively follow links
<li> Using spider arguments </li>

## Preliminaries

### Install scrapy

In [ ]:
!pip install scrapy

**Note**: If this doesn't work, see the [scrapy installation guide](https://docs.scrapy.org/en/latest/intro/install.html).

### Creating a project

Before you start scraping, you will have to set up a new Scrapy project. Enter a directory where you’d like to store your code and run:

```shell
scrapy startproject tutorial
```

This will create a tutorial directory with the following contents:

```shell
tutorial/
    scrapy.cfg            # deploy configuration file
    tutorial/             # project's Python module, you'll import your code from here
        __init__.py
        items.py          # project items definition file
        middlewares.py    # project middlewares file
        pipelines.py      # project pipelines file
        settings.py       # project settings file
        spiders/          # a directory where you'll later put your spiders
            __init__.py
            ```

## A simple scrapy spider
Spiders are classes that you define and that Scrapy uses to scrape information from a website (or a group of websites). They must subclass scrapy.Spider and define the initial requests to make, optionally how to follow links in the pages, and how to parse the downloaded page content to extract data.

This is the code for our first Spider. Save it in a file named `quotes_spider.py` under the `tutorial/spiders` directory in your project:

In [ ]:
import scrapy


class QuotesSpider(scrapy.Spider):
    name = "quotes"
    start_urls = [
        'http://quotes.toscrape.com/page/1/',
        'http://quotes.toscrape.com/page/2/',
    ]

    def parse(self, response):
        page = response.url.split("/")[-2]
        filename = 'quotes-%s.html' % page
        with open(filename, 'wb') as f:
            f.write(response.body)

As you can see, our Spider subclasses scrapy.Spider and defines some attributes and methods:

-`name`: identifies the Spider. It must be unique within a project, that is, you can’t set the same name for different Spiders.

-`start_urls`: a list of URLs to provide the initial requests for the crawler. Armed with this list alone, the spider will download HTML from the webpages specified, much as a web browser does. But it won't extract anything from the pages--that's why we need to define the `parse()` method.

-`parse()`: a method that will be called to handle the response downloaded for each of the requests made. The response parameter is an instance of `TextResponse` that holds the page content and has further helpful methods to handle it.

The `parse()` method usually parses the response, extracting the scraped data as dicts and also finding new URLs to follow and creating new requests (`Request`) from them.

### How to run the spider

To put our spider to work, go to the project’s top level directory and run:

This command runs the spider with name `quotes` that we’ve just added, that will send some requests for the `quotes.toscrape.com` domain. You will get an output similar to this:

```shell
...
2019-03-19 15:58:49 [scrapy.middleware] INFO: Enabled spider middlewares:
['scrapy.spidermiddlewares.httperror.HttpErrorMiddleware',
 'scrapy.spidermiddlewares.offsite.OffsiteMiddleware',
 'scrapy.spidermiddlewares.referer.RefererMiddleware',
 'scrapy.spidermiddlewares.urllength.UrlLengthMiddleware',
 'scrapy.spidermiddlewares.depth.DepthMiddleware']
2019-03-19 15:58:49 [scrapy.middleware] INFO: Enabled item pipelines:
[]
2019-03-19 15:58:49 [scrapy.core.engine] INFO: Spider opened
2019-03-19 15:58:49 [scrapy.extensions.logstats] INFO: Crawled 0 pages (at 0 pages/min), scraped 0 items (at 0 items/min)
2019-03-19 15:58:49 [scrapy.extensions.telnet] INFO: Telnet console listening on 127.0.0.1:6023
2019-03-19 15:58:49 [scrapy.core.engine] DEBUG: Crawled (404) <GET http://quotes.toscrape.com/robots.txt> (referer: None)
2019-03-19 15:58:49 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://quotes.toscrape.com/page/1/> (referer: None)
2019-03-19 15:58:49 [quotes] DEBUG: Saved file quotes-1.html
2019-03-19 15:58:49 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://quotes.toscrape.com/page/2/> (referer: None)
2019-03-19 15:58:50 [quotes] DEBUG: Saved file quotes-2.html
2019-03-19 15:58:50 [scrapy.core.engine] INFO: Closing spider (finished)
2019-03-19 15:58:50 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 678,
 'downloader/request_count': 3,
 ...}
2019-03-19 15:58:50 [scrapy.core.engine] INFO: Spider closed (finished)
```

Now, check the files in the current directory. You should notice that two new files have been created: `quotes-1.html` and `quotes-2.html`, with the content for the respective URLs, as our `parse` method instructs.

How did this work? Scrapy schedules the `scrapy.Request` objects returned by the `start_requests` method of the Spider. Upon receiving a response for each one, it instantiates `Response` objects and calls the callback method associated with the request (in this case, the `parse` method) passing the response as argument.

### Challenge
Modify and run the spider script above to scrape this short list of `start_urls`: 
```python
['http://brickset.com/sets/year-2016']
 ```

## Extracting data
The best way to learn how to extract data with Scrapy is trying selectors using the shell [Scrapy shell](https://docs.scrapy.org/en/latest/topics/shell.html#topics-shell). Remember to always enclose URLs in quotes (double quotes for Windows) when running Scrapy shell from command-line, otherwise urls containing arguments (ie. `&` character) will not work. Run:
```shell
scrapy shell 'http://quotes.toscrape.com'
```
You will see something like:

```shell
2019-03-19 20:00:05 [scrapy.utils.log] INFO: Scrapy 1.6.0 started (bot: tutorial)
2019-03-19 20:00:05 [scrapy.utils.log] INFO: Versions: lxml 4.3.1.0, libxml2 2.9.9, cssselect 1.0.3, parsel 1.5.1, w3lib 1.20.0, Twisted 17.9.0, Python 3.6.7 (default, Oct 22 2018, 11:32:17) - [GCC 8.2.0], pyOpenSSL 17.5.0 (OpenSSL 1.1.0g  2 Nov 2017), cryptography 2.1.4, Platform Linux-4.15.0-45-generic-x86_64-with-Ubuntu-18.04-bionic
2019-03-19 20:00:05 [scrapy.crawler] INFO: Overridden settings: {...}
2019-03-19 20:00:05 [scrapy.extensions.telnet] INFO: Telnet Password: 030319d194e7f6b0
2019-03-19 20:00:05 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage']
2019-03-19 20:00:05 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.robotstxt.RobotsTxtMiddleware',
...]
2019-03-19 20:00:05 [scrapy.middleware] INFO: Enabled spider middlewares:
['scrapy.spidermiddlewares.httperror.HttpErrorMiddleware',
...]
2019-03-19 20:00:05 [scrapy.middleware] INFO: Enabled item pipelines:
[]
2019-03-19 20:00:05 [scrapy.core.engine] INFO: Spider opened
2019-03-19 20:00:05 [scrapy.core.engine] DEBUG: Crawled (404) <GET http://quotes.toscrape.com/robots.txt> (referer: None)
2019-03-19 20:00:05 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://quotes.toscrape.com> (referer: None)
[s] Available Scrapy objects:
[s]   scrapy     scrapy module (contains scrapy.Request, scrapy.Selector, etc)
[s]   crawler    <scrapy.crawler.Crawler object at 0x7f07993c02b0>
... 
```

Using the shell, you can try selecting elements using CSS with the response object:
```shell
>>> response.css('title')
```
The result of running `response.css('title')` is a list-like object called SelectorList, which represents a list of Selector objects that wrap around XML/HTML elements and allow you to run further queries to fine-grain the selection or extract the data.

To extract the text from the title above, you can do:
```shell
>>> response.css('title::text').getall()
```

There are two things to note here: one is that we’ve added `::text` to the CSS query, to mean we want to select only the text elements directly inside `<title>` element. If we don’t specify `::text`, we’d get the full title element, including its tags:
```shell
>>> response.css('title').getall()
```
The other thing is that the result of calling `.getall()` is a list: it is possible that a selector returns more than one result, so we extract them all. When you know you just want the first result, as in this case, you can do:
```shell
>>> response.css('title::text').get()
```

Besides the `getall()` and `get()` methods, you can also use the `re()` method to extract using regular expressions:
```shell
>>> response.css('title::text').re(r'Quotes.*')
>>> response.css('title::text').re(r'Q\w+')
>>> response.css('title::text').re(r'(\w+) to (\w+)')
```
In order to find the proper CSS selectors to use, you can use your browser developer tools (e.g., in Chrome, right click > `Inspect`) to inspect the HTML and come up with a selector (for more info, see [Using your browser’s Developer Tools for scraping](https://docs.scrapy.org/en/latest/topics/developer-tools.html)). You can also try opening the response page from the shell in your web browser using `view(response)`.

### Challenge
Inspect [quotes.toscrape.com](quotes.toscrape.com) for the selectors associated with quotes. Use this information to display the text of one of the quotes in the scrapy shell. If you need help getting a better sense of website structure, use the HTML tree below as a visual guide.<br>
**Hint:** You can subset within selectors by using periods and spaces. For instance, the following produces a SelectorList for the class2 of each type2 within the class1 of each type1:
```shell
response.css('type1.class1 type2.class2')
```

### Extracting quotes and authors
Now that you know a bit about selection and extraction, let’s complete our spider by writing the code to extract the quotes from the web page.

Each quote in http://quotes.toscrape.com is represented by HTML elements that look like this:

```html
<div class="quote">
    <span class="text">“The world as we have created it is a process of our
    thinking. It cannot be changed without changing our thinking.”</span>
    <span>
        by <small class="author">Albert Einstein</small>
        <a href="/author/Albert-Einstein">(about)</a>
    </span>
    <div class="tags">
        Tags:
        <a class="tag" href="/tag/change/page/1/">change</a>
        <a class="tag" href="/tag/deep-thoughts/page/1/">deep-thoughts</a>
        <a class="tag" href="/tag/thinking/page/1/">thinking</a>
        <a class="tag" href="/tag/world/page/1/">world</a>
    </div>
</div>
```

How do we extract the data we want? To start, we get a list of selectors for the quote HTML elements with:

```shell
>>> response.css("div.quote")
```

Each of the selectors returned by the query above allows us to run further queries over their sub-elements. Let’s assign the first selector to a variable, so that we can run our CSS selectors directly on a particular quote:
```shell
>>> quote = response.css("div.quote")[0]```

Now, let’s extract title, author and the tags from that quote using the quote object we just created:

```shell
>>> title = quote.css("span.text::text").get()
>>> title
>>> author = quote.css("small.author::text").get()
>>> author
```

Given that the tags are a list of strings, we can use the .getall() method to get all of them:
```shell
>>> tags = quote.css("div.tags a.tag::text").getall()
>>> tags
```

Having figured out how to extract each bit, we can now iterate over all the quotes elements and put them together into a Python dictionary:
```shell
>>> for quote in response.css("div.quote"):
...     text = quote.css("span.text::text").get()
...     author = quote.css("small.author::text").get()
...     tags = quote.css("div.tags a.tag::text").getall()
...     print(dict(text=text, author=author, tags=tags))
>>>
```

### Extracting data in our spider
Let’s get back to our spider. Until now, it doesn’t extract any data in particular, just saves the whole HTML page to a local file. Let’s integrate the extraction logic above into our spider.

A Scrapy spider typically generates many dictionaries containing the data extracted from the page. To do that, we use the `yield` Python keyword in the callback, as you can see below:

In [ ]:
import scrapy


class QuotesSpider(scrapy.Spider):
    name = "quotes"
    start_urls = [
        'http://quotes.toscrape.com/page/1/',
        'http://quotes.toscrape.com/page/2/',
    ]

    def parse(self, response):
        for quote in response.css('div.quote'):
            yield {
                'text': quote.css('span.text::text').get(),
                'author': quote.css('small.author::text').get(),
                'tags': quote.css('div.tags a.tag::text').getall(),
            }

If you run this spider, it will output the extracted data with the log:
```shell
2016-09-19 18:57:19 [scrapy.core.scraper] DEBUG: Scraped from <200 http://quotes.toscrape.com/page/1/>
{'tags': ['life', 'love'], 'author': 'André Gide', 'text': '“It is better to be hated for what you are than to be loved for what you are not.”'}
2016-09-19 18:57:19 [scrapy.core.scraper] DEBUG: Scraped from <200 http://quotes.toscrape.com/page/1/>
{'tags': ['edison', 'failure', 'inspirational', 'paraphrased'], 'author': 'Thomas A. Edison', 'text': "“I have not failed. I've just found 10,000 ways that won't work.”"}
```

## Storing the scraped data
The simplest way to store the scraped data is by using Feed exports, with the following command:
```shell
scrapy crawl quotes -o quotes.json
```

That will generate an quotes.json file containing all scraped items, serialized in JSON.

For historic reasons, Scrapy appends to a given file instead of overwriting its contents. If you run this command twice without removing the file before the second time, you’ll end up with a long JSON file--actually, a broken JSON file, which cannot be read.

You can also use other formats, like JSON Lines:
```shell
scrapy crawl quotes -o quotes.jl
```

The JSON Lines format is useful because it’s stream-like, you can easily append new records to it. It doesn’t have the same problem of JSON when you run twice. Also, as each record is a separate line, you can process big files without having to fit everything in memory.

In small projects (like the one in this tutorial), that should be enough. However, if you want to perform more complex things with the scraped items, you can write an Item Pipeline. A placeholder file for Item Pipelines has been set up for you when the project is created, in `tutorial/pipelines.py`. Though you don’t need to implement any item pipelines if you just want to store the scraped items.

## Following links
Let’s say, instead of just scraping the stuff from the first two pages from http://quotes.toscrape.com, you want quotes from all the pages in the website.

Now that you know how to extract data from pages, let’s see how to follow links from them.

First thing is to extract the link to the page we want to follow. Examining our page, we can see there is a link to the next page with the following markup:

```shell
<ul class="pager">
    <li class="next">
        <a href="/page/2/">Next <span aria-hidden="true">&rarr;</span></a>
    </li>
</ul>
```

We can try extracting it in the shell:
```shell
>>> response.css('li.next a').get()
```

This gets the anchor element, but we want the attribute href. For that, Scrapy supports a CSS extension that lets you select the attribute contents, like this:
```shell
>>> response.css('li.next a::attr(href)').get()
```
There is also an attrib property available (see [Selecting element attributes](https://docs.scrapy.org/en/latest/topics/selectors.html#selecting-attributes) for more):
```shell
>>> response.css('li.next a').attrib['href']
```
Let’s modify our spider to recursively follow the link to the next page and extract data from it:

In [ ]:
import scrapy


class QuotesSpider(scrapy.Spider):
    name = "quotes"
    start_urls = [
        'http://quotes.toscrape.com/page/1/',
    ]

    def parse(self, response):
        for quote in response.css('div.quote'):
            yield {
                'text': quote.css('span.text::text').get(),
                'author': quote.css('small.author::text').get(),
                'tags': quote.css('div.tags a.tag::text').getall(),
            }
            
        next_page = response.css('li.next a::attr(href)').get()
        if next_page is not None:
            yield response.follow(next_page, callback=self.parse)

Now, after extracting the data, the `parse()` method looks for the link to the next page and yields a new request to the next page, registering itself as callback to handle the data extraction for the next page and to keep the crawling going through all the pages.

What you see here is Scrapy’s mechanism of following links: when you yield a `Request` in a callback method (as `response.follow` does), Scrapy will schedule that request to be sent and register a callback method to be executed when that request finishes.

Using this, you can build complex crawlers that follow links according to rules you define, and extract different kinds of data depending on the page it’s visiting.

In our example, it creates a sort of loop, following all the links to the next page until it doesn’t find one – handy for crawling blogs, forums and other sites with pagination.

## Using spider arguments
You can provide command line arguments to your spiders by using the `-a` option when running them:
```shell
scrapy crawl quotes -o quotes-humor.json -a tag=humor
```
These arguments are passed to the Spider’s `__init__` method and become spider attributes by default.

In this example, the value provided for the `tag` argument will be available via `self.tag`. You can use this to make your spider fetch only quotes with a specific tag, building the URL based on the argument:

In [ ]:
import scrapy


class QuotesSpider(scrapy.Spider):
    name = "quotes"

    def start_requests(self):
        url = 'http://quotes.toscrape.com/'
        tag = getattr(self, 'tag', None)
        if tag is not None:
            url = url + 'tag/' + tag
        yield scrapy.Request(url, self.parse)

    def parse(self, response):
        for quote in response.css('div.quote'):
            yield {
                'text': quote.css('span.text::text').get(),
                'author': quote.css('small.author::text').get(),
            }

        next_page = response.css('li.next a::attr(href)').get()
        if next_page is not None:
            yield response.follow(next_page, self.parse)

If you pass the `tag=humor` argument to this spider, you’ll notice that it will only visit URLs from the humor tag, such as http://quotes.toscrape.com/tag/humor.

You can learn more about handling spider arguments [here](https://docs.scrapy.org/en/latest/topics/spiders.html#spiderargs).

## Challenges
Here are some ways you could expand the code you've written. They'll give you some practice scraping data.

- Right now we’re only parsing results from XX, as you might have guessed from the 2016 part of `http://brickset.com/sets/year-2016` — how would you crawl results from other years?
- There's an XX included on most sets. How do you extract the data from that cell? How would you get a XX out of it? AUTHORSHIP DATA
- Most of the results have tags that specify semantic data about the sets or their context. How do we crawl these, given that there are multiple tags for a single set?